### 6.1 Working with text data

文本是最常见的序列数据之一，一般理解为单词序列。由于深度学习模型并非接收原始文本作为输入，它只能处理数值张量。因此要先将文本向量化vectorize，一般有以下几种实现方法：

- 将文本分割为单词，并将每个单词转换为一个向量
- 将文本分割为字符，并将每个字符转换为一个向量
- 提取单词或字符的n-gram，并将每个n-gram转换为一个向量。n-gram是多个相邻单词或字符的集合，n-gram之间可以重叠。

将文本分解而成的单元(单词、字符或n-gram)叫作token，将文本分解为标记的过程叫作分词(tokenization)。所有文本向量化过程都是**应用某种分词方案，然后将数值向量与生成的标记相关联。**本接介绍两种方法，one-hot encoding与token embedding。

n-gram的词袋模型是一种特征工程工具，不适用于深度学习模型，但对于轻量级的浅层文本处理模型(如logistic回归和random forest)是重要且不可或缺的。

#### 6.1.1 One-hot encoding of words and characters

one-hot编码是将token转换为向量最基本的方法。它将每个单词和一个唯一的整数index相关联，并将这个整数index转换为长度为N的二进制向量(N是词表大小)。

下面是单词级的one-hot编码和字符级的one-hot编码的简单示例：

In [1]:
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1

max_length = 10

results = np.zeros(shape=(len(samples),
                          max_length,
                          max(token_index.values())+ 1))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1

Keras中提供了内置函数，其可以对原始文本数据进行单词级或字符级的one-hot编码。这些函数一般实现了许多重要特性，应该使用它们对文本数据进行编码处理。

In [3]:
# 6-3 Using Keras for word-level one-hot encoding
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples)

# to list
sequences = tokenizer.texts_to_sequences(samples)

# to one-hot tensor
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.


one-hot encoding的一种变体是one-hot hashing trick，适用于词表中token数目太大而无法直接处理的情况。这种方法没有为每个单词显示分配一个index，而是将单词hash encode为固定长度的向量，这一步通常用一个简单的hash function实现。优点在于避免维护一个显式的单词索引，从而能够节省内存并允许数据的online encoding(在读取完所有数据前即可生成encoding)；缺点在于可能出现hash collision。

#### 6.1.2 Using word embeddings

word embedding相比于one-hot编码的词向量，具有以下特征：
- one-hot词向量稀疏、高维、hard-code
- word embedding密集、低维且从数据中学习得到

word embedding的获取有两种方法：
- 在完成主任务的同时学习该任务上的embedding。
- 在不同于待解决问题的机器学习任务上预先计算好word embedding，然后将其加载到模型中，这种word embedding叫作pretrained word embedding。

##### Learning word embedding with Embedding layer

词向量之间的几何关系应该表示这些词之间的语义关系。也即word embedding的作用应该是将人类的语言映射到几何空间中。词与词之间的distance和direction应该都是meaningful的。

对于一特定任务如需学习一个其word embeddings对应的嵌入空间，意味着要学习一个层的weights，这个层也即是Embedding层。

In [8]:
from keras.layers import Embedding

embedding_layer = Embedding(1000, 64)

Embedding层至少需要两个参数，input_dim可理解为token个数，output_dim可理解为embedding的维度。这样一来，Embedding层就可以被视为一个字典，它将单词的整数索引映射为密集向量，实际上就是一种字典查找。

这个Embedding层的input应该是一个shape为(samples, sequence_length)的张量，其中sequence_length用zero padding等方法做到长度一致。而output的shape应该是(samples, sequence_length, embedding_dimensionality)，之后可以用RNN层或者1D conv层来处理这个三维张量。

下面在IMDB的电影评论数据集上训练一个word embedding，将评论长度限制为只有20个单词，词表选择前10000个单词。

In [11]:
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000
maxlen = 20

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=
                                                     max_features)

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [12]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))

model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=10, batch_size=32,
                    validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 36s 2ms/step - loss: 0.6759 - acc: 0.6050 - val_loss: 0.6398 - val_acc: 0.6814
Epoch 2/10
20000/20000 [==============================] - 8s 415us/step - loss: 0.5657 - acc: 0.7427 - val_loss: 0.5467 - val_acc: 0.7206
Epoch 3/10
20000/20000 [==============================] - 9s 432us/step - loss

可以看到最后训练出的Embedding层的表示用于二分类任务的分类精度约为75%。但是用Dense层而非序列模型的RNN unit作分类会导致模型对输入序列中的每个单词单独处理，而并没有考虑单词之间的关系和句子结构。

##### Using pretrained word embedding

当手头可用的训练数据很少时，可从pretrained的embedding空间加载word embedding。通常这个pretrained embedding space是高度结构化的，且抓住了语言结构的一般特点。这样可以不必在解决问题的同时学习word embedding。

有许多pretrained的word embedding，都可以下载并在Keras的Embedding层中使用，如word2vec和GloVe。

#### 6.1.3 Putting it all together: from raw text to word embeddings

本节中将使用GloVe作为embedding实现IMDB数据集上的情感分类，其中IMDB的数据将使用IMDB的原始文本数据，而不是Keras内置的已经预先分词的IMDB数据。

In [ ]:
import os

imdb_dir = r''
train_dir = r''

labels = []
texts = []

for label_type in ['pos', 'neg']:
    dir_name = os.path.join()
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open